# Reduced Normal Equations

## Prerequisites

### Knoten

An [Installation of Knoten](https://github.com/DOI-USGS/knoten/tree/main#installing) is requried.
Most imports are included in the [knoten environment](https://github.com/DOI-USGS/knoten/blob/main/environment.yml).

### ISIS
An [installation of ISIS](https://astrogeology.usgs.gov/docs/how-to-guides/environment-setup-and-maintenance/installing-isis-via-anaconda/) and [ISIS-data](https://github.com/DOI-USGS/ISIS3?tab=readme-ov-file#full-isis-data-download) are also required.

`ISISROOT` and `ISISDATA` must be set to point to your ISIS and ISIS-data installations.

In [1]:
import os

# Kalasiris requires these to be set:
os.environ['ISISROOT'] = '/usr/local/Caskroom/miniconda/base/envs/isis-prod'
os.environ['ISISDATA'] = '/Volumes/usgs-data/isis-data'

from subprocess import CalledProcessError
import urllib.request

import kalasiris as isis
from plio.io import io_controlnetwork
from knoten.bundle import *

from scipy import sparse
import numpy as np

In [2]:
# File Paths

imgurl = [
    'https://planetarydata.jpl.nasa.gov/img/data/mro/mars_reconnaissance_orbiter/ctx/mrox_2130/data/F02_036648_2021_XN_22N022W.IMG',
    'https://planetarydata.jpl.nasa.gov/img/data/mro/mars_reconnaissance_orbiter/ctx/mrox_2281/data/F06_038336_2024_XN_22N022W.IMG',
    'https://planetarydata.jpl.nasa.gov/img/data/mro/mars_reconnaissance_orbiter/ctx/mrox_2479/data/F22_044336_2020_XN_22N022W.IMG',
    'https://planetarydata.jpl.nasa.gov/img/data/mro/mars_reconnaissance_orbiter/ctx/mrox_2741/data/J07_047448_2024_XN_22N022W.IMG',
]

image_dir = 'data'
imgfile = [ os.path.join(image_dir, os.path.basename(url)) for url in imgurl]

In [3]:
# Download Files

downloader = urllib.request.URLopener()

for url, file in zip(imgurl, imgfile):
    if not os.path.isfile(file):
        downloader.retrieve(url, file)

In [4]:
# Import into ISIS and SpiceInit

cubfile = [os.path.splitext(img)[0] + '.cal.cub' for img in imgfile]

try:
    for img, cub in zip(imgfile, cubfile):
        isis.mroctx2isis(img, to=cub)
except CalledProcessError as e:
    print('MROCTX2ISIS ' + e.stderr)

try:
    for cub in cubfile:
        isis.spiceinit(cub)
except CalledProcessError as e:
    print('SPICEINIT ' + e.stderr)

In [5]:
cubes = 'data/cubes.lis'
sensors = generate_sensors(cubes)

network = 'data/hand_dense.net'
cnet = io_controlnetwork.from_isis(network)
sensors = {sn: sensors[sn] for sn in cnet["serialnumber"].unique()}
cnet = compute_apriori_ground_points(cnet, sensors) # autoseed did not generate ground points, calculate and repopulate the data frame

/usr/local/Caskroom/miniconda/base/envs/knoten/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [6]:
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()}
for sn, parameters in all_parameters.items():
    print(f"Image: {sn}")
    for param in parameters:
        print(f"  {param.name} | {param.index} | {param.value}")

Image: MRO/CTX/1085197697:073
  IT Pos. Bias    | 0 | 0.0
  CT Pos. Bias    | 1 | 0.0
  Rad Pos. Bias   | 2 | 0.0
  IT Vel. Bias    | 3 | 0.0
  CT Vel. Bias    | 4 | 0.0
  Rad Vel. Bias   | 5 | 0.0
  Omega Bias      | 6 | 0.0
  Phi Bias        | 7 | 0.0
  Kappa Bias      | 8 | 0.0
  Omega Rate      | 9 | 0.0
  Phi Rate        | 10 | 0.0
  Kappa Rate      | 11 | 0.0
  Omega Accl      | 12 | 0.0
  Phi Accl        | 13 | 0.0
  Kappa Accl      | 14 | 0.0
  Focal Bias      | 15 | 0.0
Image: MRO/CTX/1157902986:250
  IT Pos. Bias    | 0 | 0.0
  CT Pos. Bias    | 1 | 0.0
  Rad Pos. Bias   | 2 | 0.0
  IT Vel. Bias    | 3 | 0.0
  CT Vel. Bias    | 4 | 0.0
  Rad Vel. Bias   | 5 | 0.0
  Omega Bias      | 6 | 0.0
  Phi Bias        | 7 | 0.0
  Kappa Bias      | 8 | 0.0
  Omega Rate      | 9 | 0.0
  Phi Rate        | 10 | 0.0
  Kappa Rate      | 11 | 0.0
  Omega Accl      | 12 | 0.0
  Phi Accl        | 13 | 0.0
  Kappa Accl      | 14 | 0.0
  Focal Bias      | 15 | 0.0
Image: MRO/CTX/1096561308:045
  

In [7]:
# Solve for angles and angular rates
solve_parameters = {sn: params[6:12] for sn, params in all_parameters.items()}
for sn, parameters in solve_parameters.items():
    print(f"Image: {sn}")
    for param in parameters:
        print(f"  {param.name} | {param.index} | {param.value}")

Image: MRO/CTX/1085197697:073
  Omega Bias      | 6 | 0.0
  Phi Bias        | 7 | 0.0
  Kappa Bias      | 8 | 0.0
  Omega Rate      | 9 | 0.0
  Phi Rate        | 10 | 0.0
  Kappa Rate      | 11 | 0.0
Image: MRO/CTX/1157902986:250
  Omega Bias      | 6 | 0.0
  Phi Bias        | 7 | 0.0
  Kappa Bias      | 8 | 0.0
  Omega Rate      | 9 | 0.0
  Phi Rate        | 10 | 0.0
  Kappa Rate      | 11 | 0.0
Image: MRO/CTX/1096561308:045
  Omega Bias      | 6 | 0.0
  Phi Bias        | 7 | 0.0
  Kappa Bias      | 8 | 0.0
  Omega Rate      | 9 | 0.0
  Phi Rate        | 10 | 0.0
  Kappa Rate      | 11 | 0.0
Image: MRO/CTX/1136952576:186
  Omega Bias      | 6 | 0.0
  Phi Bias        | 7 | 0.0
  Kappa Bias      | 8 | 0.0
  Omega Rate      | 9 | 0.0
  Phi Rate        | 10 | 0.0
  Kappa Rate      | 11 | 0.0


In [8]:
column_dict = compute_coefficient_columns(cnet, sensors, solve_parameters)

In [9]:
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations) # this is a place holder until Jesse adds his calculations
W_params = compute_parameter_weights(cnet, sensors, solve_parameters, column_dict)
dof = W_observations.shape[0] - W_params.shape[0]

In [10]:
max_iterations = 10
tol = 1e-10

In [11]:
num_sensor_params = sum([len(parameters) for parameters in solve_parameters.values()])
num_point_params = 3 * len(cnet["id"].unique())

In [12]:
sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

column_dict = compute_coefficient_columns(cnet, sensors, solve_parameters)
num_parameters = max(col_range[1] for col_range in column_dict.values())
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations)
W_params = compute_parameter_weights(cnet, sensors, solve_parameters, column_dict)

iteration = 0
V = compute_residuals(cnet, sensors)
dX = np.zeros(W_params.shape[0]) #initialize for sigma calculatioN
sigma0 = compute_sigma0(V, dX, W_params, W_observations)
print(f'iteration {iteration}: sigma0 = {sigma0}\n')

total_correction_dense = np.zeros(num_parameters)
for i in range(max_iterations):   
    iteration += 1
    old_sigma0 = sigma0
    
    J = compute_jacobian(cnet, sensors, solve_parameters, column_dict)    
    N = J.T.dot(W_observations).dot(J) + W_params # calculate the normal equation
    C = J.T.dot(W_observations).dot(V) - W_params.dot(total_correction_dense)
    dX = np.linalg.inv(N).dot(C) #calculate change in camera parameters and ground points
    total_correction_dense += dX
    print(f'corrections: mean = {dX.mean()} min = {dX.min()} max = {dX.max()}')
    
    update_parameters(sensors, solve_parameters, cnet, dX, column_dict)
    
    V = compute_residuals(cnet, sensors)
    sigma0 = compute_sigma0(V, dX, W_params, W_observations)
    sigma0 = np.sqrt((V.dot(W_observations).dot(V) + dX.dot(W_params).dot(dX))/dof)
    print(f'iteration {iteration}: sigma0 = {sigma0}\n')
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break
    

iteration 0: sigma0 = 3.386510411078402

corrections: mean = 0.02519442538434387 min = -26.658758524079243 max = 19.405572382862808
iteration 1: sigma0 = 0.8522450273089555

corrections: mean = 6.867275520848591e-05 min = -0.004864950076602206 max = 0.005917455724117964
iteration 2: sigma0 = 0.6410740414243817

corrections: mean = 1.8372002331067272e-08 min = -6.441777911409558e-07 max = 1.5574705798002814e-06
iteration 3: sigma0 = 0.6410739981011393

corrections: mean = -1.5960753684616694e-11 min = -8.058440895871958e-10 max = 8.864516184747238e-10
iteration 4: sigma0 = 0.64107399810154

change in sigma0 of 4.00679489587219e-13 converged!


In [13]:
sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

# This is setup once per bundle and then accumulates
total_corrections = np.zeros(num_sensor_params + num_point_params)

# These are computed once per bundle
W_CC_sparse = sparse.csc_matrix((num_sensor_params, num_sensor_params))
W_PP = {}

# Compute image param weight matrices
for sn, params in solve_parameters.items():
    coeff_indices = column_dict[sn]
    coeff_range = np.arange(coeff_indices[0], coeff_indices[1])
    num_image_coeffs = coeff_indices[1] - coeff_indices[0]
    W_CC_data = compute_image_weight(sensors[sn], params).ravel()
    W_CC_row = np.repeat(coeff_range, num_image_coeffs)
    W_CC_column = np.tile(coeff_range, num_image_coeffs)
    W_CC_sparse += sparse.coo_matrix((W_CC_data, (W_CC_row, W_CC_column)), (num_sensor_params, num_sensor_params)).tocsc()
    
# Compute point param weight matrices
for point_id in cnet['id'].unique():
    W_PP[point_id] = compute_point_weight(cnet, point_id)
    
V = compute_residuals(cnet, sensors)
sigma0 = compute_sigma0_sparse(V, np.zeros(total_corrections.shape), W_CC_sparse, W_PP, W_observations, column_dict)
    
# Start iteration logic
for i in range(max_iterations):
    old_sigma0 = sigma0
    
    H_CC_sparse = sparse.csc_matrix((num_sensor_params, num_sensor_params))
    H_CC_sparse += W_CC_sparse

    g_C_sparse = np.zeros(num_sensor_params)
    g_C_sparse -= W_CC_sparse.dot(total_corrections[:num_sensor_params])
    # g_C_sparse += W_CC_sparse.dot(sensor_corrections)

    # Q = H_PP^-1 * H_PC 
    Q_mats = {}
    # NIC = H_PP^-1 * g_P
    NIC_vecs = {}

    updates = np.zeros(num_sensor_params + num_point_params)

    for point_id, group in cnet.groupby('id'):
        ground_pt = group.iloc[0][["adjustedX", "adjustedY", "adjustedZ"]]
        H_CP = sparse.csc_matrix((num_sensor_params, 3))
        H_PP = np.zeros((3, 3))
        g_P = np.zeros(3)
        for measure_idx, row in group.iterrows():
            serial = row["serialnumber"]
            sensor = sensors[serial]
            point_partials = compute_ground_partials(sensor, ground_pt)
            sensor_partials = compute_sensor_partials(sensor, solve_parameters[serial], ground_pt)
            coeff_indices = column_dict[serial]
            coeff_range = np.arange(coeff_indices[0], coeff_indices[1])
            num_image_coeffs = coeff_indices[1] - coeff_indices[0]

            H_CC_point_data = np.dot(sensor_partials.T, sensor_partials).ravel()
            H_CC_point_row = np.repeat(coeff_range, num_image_coeffs)
            H_CC_point_column = np.tile(coeff_range, num_image_coeffs)
            H_CC_sparse += sparse.coo_matrix((H_CC_point_data, (H_CC_point_row, H_CC_point_column)), (num_sensor_params, num_sensor_params)).tocsc()

            H_CP_point_data = np.dot(sensor_partials.T, point_partials).ravel()
            H_CP_point_row = np.repeat(coeff_range, 3)
            H_CP_point_column = np.tile(np.arange(0, 3), num_image_coeffs)
            H_CP += sparse.coo_matrix((H_CP_point_data, (H_CP_point_row, H_CP_point_column)), (num_sensor_params, 3)).tocsc()

            H_PP += np.dot(point_partials.T, point_partials)

            g_C_sparse[coeff_indices[0]:coeff_indices[1]] += np.dot(sensor_partials.T, V[2*measure_idx:2*measure_idx+2])
            g_P += np.dot(point_partials.T, V[2*measure_idx:2*measure_idx+2])
        point_param_range = column_dict[point_id]
        g_P -= W_PP[point_id].dot(total_corrections[point_param_range[0]:point_param_range[1]])
        H_PP += W_PP[point_id]
        H_PP_inv = sparse.csc_matrix(np.linalg.inv(H_PP))
        Q_mats[point_id] = H_PP_inv.dot(H_CP.transpose())
        NIC_vecs[point_id] = H_PP_inv.dot(g_P)
        H_CC_sparse -= H_CP.dot(Q_mats[point_id])
        g_C_sparse -= H_CP.dot(NIC_vecs[point_id])

    updates[:num_sensor_params] = sparse.linalg.spsolve(H_CC_sparse, g_C_sparse)

    for point_id in Q_mats:
        point_param_indices = column_dict[point_id]
        updates[point_param_indices[0]:point_param_indices[1]] = NIC_vecs[point_id] - Q_mats[point_id].dot(updates[:num_sensor_params])
        
    print(f'corrections: mean = {updates.mean()} min = {updates.min()} max = {updates.max()}')

    total_corrections += updates

    update_parameters(sensors, solve_parameters, cnet, updates, column_dict)
    V = compute_residuals(cnet, sensors)
    sigma0 = compute_sigma0_sparse(V, updates, W_CC_sparse, W_PP, W_observations, column_dict)
    print(f'iteration {i+1}: sigma0 = {sigma0}\n')
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break

corrections: mean = 0.02519442538434383 min = -26.65875852407914 max = 19.40557238286281
iteration 1: sigma0 = 0.8522450273105413

corrections: mean = 6.86727606479218e-05 min = -0.004864950150258623 max = 0.005917455863802144


iteration 2: sigma0 = 0.64107404140547

corrections: mean = 1.8352119712705532e-08 min = -6.442099360416587e-07 max = 1.5574976963027463e-06


iteration 3: sigma0 = 0.641073998118407

corrections: mean = 1.1279955347085866e-11 min = -1.0020180255690482e-09 max = 7.350010298342555e-10


iteration 4: sigma0 = 0.6410739980892225

change in sigma0 of 2.918454367062395e-11 converged!


In [14]:
print("Sensor diff")
print(np.min(total_correction_dense[:num_sensor_params].flatten() - total_corrections[:num_sensor_params]))
print(np.max(total_correction_dense[:num_sensor_params].flatten() - total_corrections[:num_sensor_params]))
print("Point diff")
print(np.min(total_correction_dense[num_sensor_params:].flatten() - total_corrections[num_sensor_params:]))
print(np.max(total_correction_dense[num_sensor_params:].flatten() - total_corrections[num_sensor_params:]))

Sensor diff
-8.027561948509288e-10
1.0439631381586878e-09
Point diff
-3.0175491272377286e-10
1.560850754200871e-10
